Intial Preprocessing of Data to get required format

In [ ]:
import pandas as pd

# Step 1: Upload Excel file
from google.colab import files
uploaded = files.upload()

Saving Company.xlsx to Company.xlsx


In [ ]:
# Step 2: Load Excel into a DataFrame
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

In [ ]:
# Step 3 (Updated): Normalize 'Round' column by replacing ':' with '-' before splitting
df['Round'] = df['Round'].str.replace(':', '-', regex=False)

# Now split into Round Number and Round Name
df[['Round Number', 'Round Name']] = df['Round'].str.split('-', n=1, expand=True)
df['Round Number'] = df['Round Number'].str.strip()
df['Round Name'] = df['Round Name'].str.strip()

In [ ]:
# Step 4 (Revised): Split 'Topic name' on '?', keep only valid non-empty questions
df['Topic Questions'] = df['Topic name'].str.replace('\n', ' ', regex=True)  # remove newlines inside text
df['Topic Questions'] = df['Topic Questions'].str.split(r'\?')  # split on question marks

# Add '?' back to each question (except empty ones)
df['Topic Questions'] = df['Topic Questions'].apply(
    lambda qlist: [q.strip() + '?' for q in qlist if q.strip()]
)

In [ ]:
# Step 5 (Updated): Explode into individual rows and strip spaces
df_exploded = df.explode('Topic Questions').reset_index(drop=True)
df_exploded['Topic Questions'] = df_exploded['Topic Questions'].str.strip()

In [ ]:
# Step 6: Clean leading commas and extra spaces from 'Topic Questions'
df_exploded['Topic Questions'] = df_exploded['Topic Questions'].str.lstrip(', ').str.strip()

In [ ]:
# Step 7: Drop original columns if needed and reset index
df_final = df_exploded.drop(columns=['Round', 'Topic name']).reset_index(drop=True)

In [ ]:
# Step 8: Display the final DataFrame
from IPython.display import display
display(df_final)

,Company,Role,Tags,Round Number,Round Name,Topic Questions
0,Meta,Machine Learning Engineer,"Behavioral,Machinelearning,Programminglanguage...",Round 1,HR Interview,Why do you want to join Meta?
1,Meta,Machine Learning Engineer,"Behavioral,Machinelearning,Programminglanguage...",Round 1,HR Interview,Why do you think you will be a good fit for th...
2,Meta,Machine Learning Engineer,"Behavioral,Machinelearning,Programminglanguage...",Round 1,HR Interview,What responsibilities do you expect to have fr...
3,Meta,Machine Learning Engineer,"Behavioral,Machinelearning,Programminglanguage...",Round 1,HR Interview,What are your areas of expertise in machine le...
4,Meta,Machine Learning Engineer,"Behavioral,Machinelearning,Programminglanguage...",Round 1,HR Interview,What programming languages are you comfortable...
...,...,...,...,...,...,...
652,Apple,Data Engineer,"deep-dive, project walkthrough, practical, cro...",Round 5,On-Site Interviews,How do you keep up with the latest development...
653,Apple,Data Engineer,"deep-dive, project walkthrough, practical, cro...",Round 5,On-Site Interviews,Can you give an example of a time when you had...
654,Apple,Data Engineer,"deep-dive, project walkthrough, practical, cro...",Round 5,On-Site Interviews,How do you prioritize and manage your workload...
655,Apple,Data Engineer,"deep-dive, project walkthrough, practical, cro...",Round 5,On-Site Interviews,Can you describe a time when you had to work w...


In [ ]:
# Optional: Save to Excel or CSV
df_final.to_excel("processed_interview_data.xlsx", index=False)
files.download("processed_interview_data.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install -q sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.4 MB/s eta 0:00:00


EMBEDDINGS

In [ ]:
# Step 1: Upload the processed Excel file
from google.colab import files
uploaded = files.upload()

# Step 2: Load Excel
import pandas as pd
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

# Step 3: Clean whitespace in key columns
for col in ['Company', 'Role', 'Round Name', 'Topic Questions']:
    df[col] = df[col].astype(str).str.strip()

# Step 4: Create combined text for embedding
df['combined_text'] = df.apply(
    lambda row: f"{row['Company']} - {row['Role']} - {row['Round Name']} - {row['Topic Questions']}",
    axis=1
)

# Step 5: Install and load embedding model
!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer
import json

model = SentenceTransformer('all-MiniLM-L6-v2')

# Step 6: Generate embeddings
df['embedding'] = df['combined_text'].apply(lambda x: model.encode(x).tolist())

# Step 7: Drop combined text (optional)
df.drop(columns=['combined_text'], inplace=True)

# Step 8: Convert to records and save as JSONL
jsonl_file = 'processed_interview_with_embeddings.jsonl'
with open(jsonl_file, 'w') as f:
    for record in df.to_dict(orient='records'):
        f.write(json.dumps(record) + '\n')

# Step 9: Download the JSONL file
files.download(jsonl_file)


Saving processed_interview_data-4.xlsx to processed_interview_data-4 (1).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Read and preview JSONL file in Colab
import json

with open('processed_interview_with_embeddings.jsonl', 'r') as f:
    lines = f.readlines()

# Parse first few JSON objects
for i, line in enumerate(lines[:5]):
    data = json.loads(line)
    print(f"\n--- Record {i+1} ---")
    print(json.dumps(data, indent=2))



--- Record 1 ---
{
  "Company": "Meta",
  "Role": "Machine Learning Engineer",
  "Tags": "Behavioral,Machinelearning,Programminglanguages,Frameworks",
  "Round Number": "Round 1",
  "Round Name": "HR Interview",
  "Topic Questions": "Why do you want to join Meta?",
  "embedding": [
    -0.028817638754844666,
    0.0369221568107605,
    0.01961103081703186,
    0.04577894136309624,
    0.15243588387966156,
    -0.045289553701877594,
    0.041193582117557526,
    0.013343757018446922,
    -0.04114454612135887,
    -0.06190160661935806,
    -0.16111840307712555,
    -0.05112757161259651,
    -0.06963227689266205,
    -0.03054792620241642,
    0.015425530262291431,
    -0.00513047631829977,
    -0.016193419694900513,
    -0.07248611003160477,
    -0.027303030714392662,
    -0.061931610107421875,
    -0.026261569932103157,
    -0.03894275426864624,
    0.04584980756044388,
    -0.04480805620551109,
    -0.04424097388982773,
    0.039501339197158813,
    -0.01852838322520256,
    0.08459563

In [ ]:
# Step 1: Install latest ChromaDB
!pip install -q chromadb sentence-transformers

# Step 2: Import updated Chroma modules
import chromadb
from chromadb.config import Settings
import json
import os

# Step 3: Initialize Chroma PersistentClient
persist_dir = "./chroma_db"
os.makedirs(persist_dir, exist_ok=True)

from chromadb import PersistentClient
client = PersistentClient(path=persist_dir)

# Step 4: Create or get a collection
collection = client.get_or_create_collection(name="interview_questions")

# Step 5: Load JSONL file
records = []
with open('processed_interview_with_embeddings.jsonl', 'r') as f:
    for line in f:
        record = json.loads(line)
        records.append(record)

# Step 6: Prepare and add data
texts = [r['Topic Questions'] for r in records]
embeddings = [r['embedding'] for r in records]
metadatas = [{k: v for k, v in r.items() if k not in ['embedding', 'Topic Questions']} for r in records]

collection.add(
    documents=texts,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=[f"q_{i}" for i in range(len(texts))]
)


In [ ]:
!pip install -q chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 10.4 MB/s eta 0

In [ ]:
# Using Gemini to extract interview details.
import google.generativeai as genai
import json, re
import chromadb
from chromadb.config import Settings

# 1. Configure Gemini + Chroma
genai.configure(REDACTED)
model = genai.GenerativeModel('gemini-1.5-flash')

from chromadb import PersistentClient
persist_dir = "./chroma_db"
client = PersistentClient(path=persist_dir)
collection = client.get_collection(name="interview_questions")


# 2. Metadata extractor using Gemini prompt
def extract_metadata_from_query():
    user_query = input("Enter your interview query: ")
    prompt = f"""
Extract the following information from the sentence below:

Sentence: "{user_query}"

Return a JSON object with:
- Company
- Role
- Round Number

Format:
{{
  "Company": "...",
  "Role": "...",
  "Round Number": "..."
}}
"""
    try:
        response = model.generate_content(prompt)
        response_json = response.text.strip()
        match = re.search(r'\{.*\}', response_json, re.DOTALL)
        if match:
            metadata = json.loads(match.group())
            print(" Extracted Metadata:", metadata)
            return metadata
        else:
            print("Could not find valid JSON in response.")
            return None
    except Exception as e:
        print("Error extracting metadata:", e)
        return None

In [ ]:
# 3. Fetching relevant docs from Vector DB
def run_full_pipeline():
    metadata = extract_metadata_from_query()
    if not metadata:
        return

    # Normalize round number
    round_number = metadata["Round Number"]
    if not round_number.lower().startswith("round"):
        round_number = f"Round {round_number}"

    metadata_filter = {
        "$and": [
            {"Company": {"$eq": metadata["Company"]}},
            {"Role": {"$eq": metadata["Role"]}},
            {"Round Number": {"$eq": round_number}}
        ]
    }

    query_string = f"interview questions for {metadata['Role']} {round_number}"
    print("\n🔍 Querying:", query_string)

    results = collection.query(
        query_texts=[query_string],
        n_results=5,
        where=metadata_filter
    )

    if not results['documents'][0]:
        print(" No results found.")
        return

    for i in range(len(results['documents'][0])):
        print(f"\n--- Match {i+1} ---")
        print("Question:", results['documents'][0][i])
        print("Metadata:", results['metadatas'][0][i])

In [ ]:
run_full_pipeline()

Enter your interview query: Could you help me practice for Data Scientist Round 3 interview for Meta
 Extracted Metadata: {'Company': 'Meta', 'Role': 'Data Scientist', 'Round Number': '3'}

🔍 Querying: interview questions for Data Scientist Round 3

--- Match 1 ---
Question: Which data sets would you use to answer specific product-related questions?
Metadata: {'Round Name': 'Analytical Reasoning Interview', 'Role': 'Data Scientist', 'Tags': 'Behavioural,Statistics,ProductCase,ExperimentDesign,BiasDetection,DataInterpretation,Storytelling,Metrics', 'Company': 'Meta', 'Round Number': 'Round 3'}

--- Match 2 ---
Question: Given a dataset on user engagement, how would you structure a solution to improve user interaction on our platform?
Metadata: {'Tags': 'Behavioural,Statistics,ProductCase,Sql,Coding,EdgeCases,DataAdaptation,CodeOptimization,Communication,Metrics', 'Round Number': 'Round 3', 'Role': 'Data Scientist', 'Round Name': 'Technical Skills Interview', 'Company': 'Meta'}

--- Matc